In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.6 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0608.csv		       Github
 a0609.csv		       heatmap529_UK.ipynb
 ccc_compiled.csv	       heatmap530.ipynb
 csv			       Heat_Map_Online_language.ipynb
 csv.zip		       __MACOSX
 dataframe_csv		       mapping_UK
 dataframe_csv_EU	       mapping_US
 dataframe_csv_UK	       non_violant1.csv
 dataframe_result_529_UK.csv   non_violant2.csv
 dataframe_result_530_UK.csv   UK
 dataframe_result_531_UK.csv   violant.csv
 dataframe_result_csv	      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct=pd.read_csv('a0529-a0611_UK/a0606.csv',encoding='ISO-8859-1')

In [8]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0608.csv		       Github
 a0609.csv		       heatmap529_UK.ipynb
 ccc_compiled.csv	       heatmap530.ipynb
 csv			       Heat_Map_Online_language.ipynb
 csv.zip		       __MACOSX
 dataframe_csv		       mapping_UK
 dataframe_csv_EU	       mapping_US
 dataframe_csv_UK	       non_violant1.csv
 dataframe_result_529_UK.csv   non_violant2.csv
 dataframe_result_530_UK.csv   UK
 dataframe_result_531_UK.csv   violant.csv
 dataframe_result_csv	      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [9]:
###
#ct=pd.read_csv('a0601.csv')

In [10]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,1204666772916238_3022436634472567,Facebook,2020-06-06 23:59:56,2021-08-27 16:21:20,link,George Floyd and officer who knelt on his neck...,nbcnews.com,The club's former owner said it appeared to he...,Trash Comes In All Colors ! Bad Boy Floyd Die ...,3.0,https://nbcnews.com/news/us-news/george-floyd-...,https://www.facebook.com/groups/12046667729162...,5393,-0.375000,1.0,3,6,1,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,0,1,4946904,Conservative Republican - People's Grassroots ...,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,NaN,NaN,False,NaN,en,0,4946904|3022436634472567,{},{},{}
1,1,987673694762272_1437498816446422,Facebook,2020-06-06 23:59:56,2021-06-26 03:00:27,photo,George Floyd and officer who knelt on his neck...,nbcnews.com,The club's former owner said it appeared to he...,it.s not about race it.s about the man just th...,1.0,https://www.facebook.com/photo.php?fbid=157715...,https://www.facebook.com/groups/98767369476227...,1321,-1.136364,1.0,9,0,13,0,0,0,0,0,0,0,3,2,4,3,4,4,3,2,0,2,10909159,The Gem Of The Peak,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_group,NaN,NaN,False,NaN,en,0,10909159|1437498816446422,{},{},{}
2,2,56845382910_10158226273137911,Facebook,2020-06-06 23:59:55,2021-06-20 11:12:15,link,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...","""Iâll step out again, even if this time ends...",1.0,http://huffp.st/hhagApZ,https://www.facebook.com/56845382910/posts/101...,2173245,4.365497,1.0,791,184,186,207,55,15,50,5,0,135,123,22,118,18,8,27,6,20,0,3,13844,HuffPost Politics,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,The latest political news from HuffPost's Wash...,True,2009-03-16 19:03:21,en,0,13844|10158226273137911,{},{},{}
3,3,457604514282180_4110753105633951,Facebook,2020-06-06 23:59:54,2021-08-27 22:26:32,photo,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...","16.25 miles for Kari, and 16 for me! (A little...",1.0,https://www.facebook.com/photo.php?fbid=101572...,https://www.facebook.com/groups/45760451428218...,1233,-1.352941,1.0,31,0,0,3,0,0,0,0,0,0,29,3,4,3,3,4,0,0,0,1,10054545,Run Minnesota Polar Bears,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,The latest political news from HuffPost's Wash...,False,2009-03-16 19:03:21,en,0,10054545|4110753105633951,{},{},{}
4,4,1425803351055421_2372418579727222,Facebook,2020-06-06 23:59:51,2021-07-20 01:35:07,status,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...",Some of you may remember Allen Payne who was a...,1.0,https://www.facebook.com/photo.php?fbid=101572...,https://www.facebook.com/groups/14258033510554...,1691,-5.833333,1.0,4,0,2,0,0,0,0,0,0,0,9,2,10,2,2,3,5,2,0,1,10933711,Xavier Musketeer Fans,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,The latest political news from HuffPost's Wash...,False,2009-03-16 19:03:21,en,0,10933711|2372418579727222,{},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [11]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [12]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [13]:
###
ct_ner = ct

In [14]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{},{}
1,{},{},{}
2,{},{},{}
3,{},{},{}
4,{},{},{}


In [15]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [16]:
ct_ner['NER-msg']

0                                                       []
1                                                       []
2                                                       []
3                                                       []
4                                                       []
                               ...                        
23248    [minneapolis, Orange County, Florida, Winderme...
23249                                                   []
23250                                           [Cornwall]
23251                                                   []
23252                     [the United States, Minneapolis]
Name: NER-msg, Length: 23253, dtype: object

In [17]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [18]:
ct_ner['NER']

0                                                       []
1                                                       []
2                                                       []
3                                                       []
4                                                       []
                               ...                        
23248    [minneapolis, Orange County, Florida, Winderme...
23249    [Minneapolis, Iraq, D.C., Lafayette Square, DC...
23250                                 [Cornwall, Cornwall]
23251                                                   []
23252    [the United States, Minneapolis, the United St...
Name: NER, Length: 23253, dtype: object

In [19]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [20]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [21]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,1204666772916238_3022436634472567,Facebook,2020-06-06 23:59:56,2021-08-27 16:21:20,link,George Floyd and officer who knelt on his neck...,nbcnews.com,The club's former owner said it appeared to he...,Trash Comes In All Colors ! Bad Boy Floyd Die ...,3.0,https://nbcnews.com/news/us-news/george-floyd-...,https://www.facebook.com/groups/12046667729162...,5393,-0.375000,1.0,3,6,1,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,0,1,4946904,Conservative Republican - People's Grassroots ...,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,NaN,NaN,False,NaN,en,0,4946904|3022436634472567,[],[],[],[]
1,1,987673694762272_1437498816446422,Facebook,2020-06-06 23:59:56,2021-06-26 03:00:27,photo,George Floyd and officer who knelt on his neck...,nbcnews.com,The club's former owner said it appeared to he...,it.s not about race it.s about the man just th...,1.0,https://www.facebook.com/photo.php?fbid=157715...,https://www.facebook.com/groups/98767369476227...,1321,-1.136364,1.0,9,0,13,0,0,0,0,0,0,0,3,2,4,3,4,4,3,2,0,2,10909159,The Gem Of The Peak,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_group,NaN,NaN,False,NaN,en,0,10909159|1437498816446422,[],[],[],[]
2,2,56845382910_10158226273137911,Facebook,2020-06-06 23:59:55,2021-06-20 11:12:15,link,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...","""Iâll step out again, even if this time ends...",1.0,http://huffp.st/hhagApZ,https://www.facebook.com/56845382910/posts/101...,2173245,4.365497,1.0,791,184,186,207,55,15,50,5,0,135,123,22,118,18,8,27,6,20,0,3,13844,HuffPost Politics,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,The latest political news from HuffPost's Wash...,True,2009-03-16 19:03:21,en,0,13844|10158226273137911,[],[],[],[]
3,3,457604514282180_4110753105633951,Facebook,2020-06-06 23:59:54,2021-08-27 22:26:32,photo,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...","16.25 miles for Kari, and 16 for me! (A little...",1.0,https://www.facebook.com/photo.php?fbid=101572...,https://www.facebook.com/groups/45760451428218...,1233,-1.352941,1.0,31,0,0,3,0,0,0,0,0,0,29,3,4,3,3,4,0,0,0,1,10054545,Run Minnesota Polar Bears,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,The latest political news from HuffPost's Wash...,False,2009-03-16 19:03:21,en,0,10054545|4110753105633951,[],[],[],[]
4,4,1425803351055421_2372418579727222,Facebook,2020-06-06 23:59:51,2021-07-20 01:35:07,status,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...",Some of you may remember Allen Payne who was a...,1.0,https://www.facebook.com/photo.php?fbid=101572...,https://www.facebook.com/groups/14258033510554...,1691,-5.833333,1.0,4,0,2,0,0,0,0,0,0,0,9,2,10,2,2,3,5,2,0,1,10933711,Xavier Musketeer Fans,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,The latest political news from HuffPost's Wash...,False,2009-03-16 19:03:21,en,0,10933711|2372418579727222,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [22]:
ct_ner['NER-1']=ct_ner['NER']

In [23]:
ct_ner['NER-1'][1]

[]

In [24]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 11.7 MB/s 


In [25]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [26]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,1204666772916238_3022436634472567,Facebook,2020-06-06 23:59:56,2021-08-27 16:21:20,link,George Floyd and officer who knelt on his neck...,nbcnews.com,The club's former owner said it appeared to he...,Trash Comes In All Colors ! Bad Boy Floyd Die ...,3.0,https://nbcnews.com/news/us-news/george-floyd-...,https://www.facebook.com/groups/12046667729162...,5393,-0.375000,1.0,3,6,1,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,0,1,4946904,Conservative Republican - People's Grassroots ...,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,NaN,NaN,False,NaN,en,0,4946904|3022436634472567,[],[],[],[],[]
1,1,987673694762272_1437498816446422,Facebook,2020-06-06 23:59:56,2021-06-26 03:00:27,photo,George Floyd and officer who knelt on his neck...,nbcnews.com,The club's former owner said it appeared to he...,it.s not about race it.s about the man just th...,1.0,https://www.facebook.com/photo.php?fbid=157715...,https://www.facebook.com/groups/98767369476227...,1321,-1.136364,1.0,9,0,13,0,0,0,0,0,0,0,3,2,4,3,4,4,3,2,0,2,10909159,The Gem Of The Peak,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_group,NaN,NaN,False,NaN,en,0,10909159|1437498816446422,[],[],[],[],[]
2,2,56845382910_10158226273137911,Facebook,2020-06-06 23:59:55,2021-06-20 11:12:15,link,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...","""Iâll step out again, even if this time ends...",1.0,http://huffp.st/hhagApZ,https://www.facebook.com/56845382910/posts/101...,2173245,4.365497,1.0,791,184,186,207,55,15,50,5,0,135,123,22,118,18,8,27,6,20,0,3,13844,HuffPost Politics,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,The latest political news from HuffPost's Wash...,True,2009-03-16 19:03:21,en,0,13844|10158226273137911,[],[],[],[],[]
3,3,457604514282180_4110753105633951,Facebook,2020-06-06 23:59:54,2021-08-27 22:26:32,photo,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...","16.25 miles for Kari, and 16 for me! (A little...",1.0,https://www.facebook.com/photo.php?fbid=101572...,https://www.facebook.com/groups/45760451428218...,1233,-1.352941,1.0,31,0,0,3,0,0,0,0,0,0,29,3,4,3,3,4,0,0,0,1,10054545,Run Minnesota Polar Bears,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,The latest political news from HuffPost's Wash...,False,2009-03-16 19:03:21,en,0,10054545|4110753105633951,[],[],[],[],[]
4,4,1425803351055421_2372418579727222,Facebook,2020-06-06 23:59:51,2021-07-20 01:35:07,status,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...",Some of you may remember Allen Payne who was a...,1.0,https://www.facebook.com/photo.php?fbid=101572...,https://www.facebook.com/groups/14258033510554...,1691,-5.833333,1.0,4,0,2,0,0,0,0,0,0,0,9,2,10,2,2,3,5,2,0,1,10933711,Xavier Musketeer Fans,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,The latest political news from HuffPost's Wash...,False,2009-03-16 19:03:21,en,0,10933711|2372418579727222,[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [27]:
%cd mapping_UK

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_UK


In [28]:
!ls

dataframe0529_post.csv	    dataframe_mapping_0603.csv
dataframe0530_post.csv	    dataframe_mapping_0604.csv
dataframe0531_post.csv	    dataframe_mapping_0605.csv
dataframe0601_post.csv	    dataframe_mapping_0606.csv
dataframe0602_post.csv	    dataframe_mapping_0607.csv
dataframe_mapping_0529.csv  dataframe_mapping_0608.csv
dataframe_mapping_0530.csv  dataframe_mapping_0609.csv
dataframe_mapping_0531.csv  dataframe_mapping_0610.csv
dataframe_mapping_0601.csv  dataframe_mapping_0611.csv
dataframe_mapping_0602.csv


In [29]:
ct_mapping=pd.read_csv('dataframe_mapping_0606.csv',encoding='ISO-8859-1')

In [30]:
len(ct_mapping['City'])

3621

In [31]:
len(ct_ner['NER-1'])

23253

In [32]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [33]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,1204666772916238_3022436634472567,Facebook,2020-06-06 23:59:56,2021-08-27 16:21:20,link,George Floyd and officer who knelt on his neck...,nbcnews.com,The club's former owner said it appeared to he...,Trash Comes In All Colors ! Bad Boy Floyd Die ...,3.0,https://nbcnews.com/news/us-news/george-floyd-...,https://www.facebook.com/groups/12046667729162...,5393,-0.375000,1.0,3,6,1,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,0,1,4946904,Conservative Republican - People's Grassroots ...,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,NaN,NaN,False,NaN,en,0,4946904|3022436634472567,[],[],[],[],[]
1,1,987673694762272_1437498816446422,Facebook,2020-06-06 23:59:56,2021-06-26 03:00:27,photo,George Floyd and officer who knelt on his neck...,nbcnews.com,The club's former owner said it appeared to he...,it.s not about race it.s about the man just th...,1.0,https://www.facebook.com/photo.php?fbid=157715...,https://www.facebook.com/groups/98767369476227...,1321,-1.136364,1.0,9,0,13,0,0,0,0,0,0,0,3,2,4,3,4,4,3,2,0,2,10909159,The Gem Of The Peak,NaN,https://scontent-sea1-1.xx.fbcdn.net/v/t39.308...,facebook_group,NaN,NaN,False,NaN,en,0,10909159|1437498816446422,[],[],[],[],[]
2,2,56845382910_10158226273137911,Facebook,2020-06-06 23:59:55,2021-06-20 11:12:15,link,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...","""Iâll step out again, even if this time ends...",1.0,http://huffp.st/hhagApZ,https://www.facebook.com/56845382910/posts/101...,2173245,4.365497,1.0,791,184,186,207,55,15,50,5,0,135,123,22,118,18,8,27,6,20,0,3,13844,HuffPost Politics,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,The latest political news from HuffPost's Wash...,True,2009-03-16 19:03:21,en,0,13844|10158226273137911,[],[],[],[],[]
3,3,457604514282180_4110753105633951,Facebook,2020-06-06 23:59:54,2021-08-27 22:26:32,photo,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...","16.25 miles for Kari, and 16 for me! (A little...",1.0,https://www.facebook.com/photo.php?fbid=101572...,https://www.facebook.com/groups/45760451428218...,1233,-1.352941,1.0,31,0,0,3,0,0,0,0,0,0,29,3,4,3,3,4,0,0,0,1,10054545,Run Minnesota Polar Bears,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,The latest political news from HuffPost's Wash...,False,2009-03-16 19:03:21,en,0,10054545|4110753105633951,[],[],[],[],[]
4,4,1425803351055421_2372418579727222,Facebook,2020-06-06 23:59:51,2021-07-20 01:35:07,status,Iâm Immunocompromised. Marching For George F...,huffpost.com,"""Iâll step out again, even if this time ends...",Some of you may remember Allen Payne who was a...,1.0,https://www.facebook.com/photo.php?fbid=101572...,https://www.facebook.com/groups/14258033510554...,1691,-5.833333,1.0,4,0,2,0,0,0,0,0,0,0,9,2,10,2,2,3,5,2,0,1,10933711,Xavier Musketeer Fans,HuffPostPolitics,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_group,US,The latest political news from HuffPost's Wash...,False,2009-03-16 19:03:21,en,0,10933711|2372418579727222,[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [34]:
ct_ner['NER-1'][12]

['San Jose, Santa Clara County, California, United States',
 'Washington, District of Columbia, United States',
 'Washington, District of Columbia, United States']

In [35]:
ct_ner.to_csv('dataframe0606_post.csv')